In [1]:
import numpy as np
import pandas as pd
import re
import glob #this makes a list of all the files in a directory
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import *

In [2]:
with_gender = pd.read_pickle("WithGenderDF2.p")
with_gender['gender'].value_counts()
line_column=[]
for i in with_gender.index:
        line_column.append(' '.join(with_gender['Speech'][i]))
with_gender['Line']=line_column

with_gender['linelength']=with_gender['Line'].str.len()
with_gender_dum = pd.get_dummies(with_gender['gender'])
with_gender = pd.concat([with_gender, with_gender_dum],axis=1).drop(['F','gender'],1)

In [5]:
#didn't end up using the word2vec
#import gensim

In [4]:
#didn't need to split the words if not vectorizing
#with_gender['words']=with_gender['Line'].str.split()
#with_gender.head()

In [7]:
#model = gensim.models.Word2Vec(with_gender['words'], min_count=1)
#we already trained this, just load it below

In [8]:
#model.save('trainedwords')

In [36]:
#model = gensim.models.Word2Vec.load('trainedwords')

In [16]:
#word_vectors = model.wv
#del model

In [5]:
#vec_column = []
#for i in with_gender.index:
#    word_vecs=[]
#    for j in with_gender['words'][i]:
#        word_vecs.append(word_vectors[j])
#    vec_column.append(word_vecs)
#with_gender['wordvecs']=vec_column
#with_gender.head()

In [2]:
#vectorized = with_gender
#vectorized.to_pickle('vectorized_lines.p')
#vectorized=pd.read_pickle('vectorized_lines.p')

In [3]:
with_gender_shuffle = shuffle(with_gender, random_state = 5)


# Randomly split into training and dev with scikit-learn helper function
train, test = train_test_split(with_gender_shuffle, test_size=0.2, random_state=100)
dev, test = train_test_split(test, test_size=0.5, random_state=200)

print train.shape
print dev.shape
print test.shape

(215353, 9)
(26919, 9)
(26920, 9)


In [4]:
train.head(n=2)

,Char,Title,Genre,pkey,LineID,Speech,Line,linelength,M
175478,KELLY,battleofshakerheightsthe,Romance,kelly-battleofshakerheightsthe,319,"[Sure. I mean, I'll help.]","Sure. I mean, I'll help.",24,1
142368,GALEN,mud,Drama,galen-mud,754,"[Neck looks up to you. Don't get my, nephew in...",Neck looks up to you. Don't get my nephew into...,81,1


In [5]:
train_y, dev_y, test_y = train.M, dev.M, test.M
train_X = train.drop(['M','Speech','LineID','pkey','linelength'],1)
dev_X = dev.drop(['M','Speech','LineID','pkey','linelength'],1)
test_X = test.drop(['M','Speech','LineID','pkey','linelength'],1)
dev_X.head()

,Char,Title,Genre,Line
224989,REILLY,ticker,Crime,I want to work this.
33696,HEROINE,feast,Horror,(to Boss Man) You said you had a short wave ra...
218517,PLATO,rebelwithoutacause,Drama,I've got a gun.
189648,MALAHIDE,hellboundhellraiserii,Horror,...are the optical motor nerve control centers.
186370,LOBRUTO,fracture,Crime,What is that - some kind of Oklahoma insult?


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.utils import np_utils
from keras.layers import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [7]:
from keras.preprocessing.text import Tokenizer
t=Tokenizer(num_words=10000)
t.fit_on_texts(train_X.Line.values)

x_train = t.texts_to_sequences(train_X.Line.values)
x_test = t.texts_to_sequences(test_X.Line.values)
x_dev = t.texts_to_sequences(dev_X.Line.values)

In [8]:
print train_X.Line.values[1]
print x_train[1]

print train_X.Line.values[2]
print x_train[2]

print dev_X.Line.values[1]
print x_dev[1]

print dev_X.Line.values[2]
print x_dev[2]

Neck looks up to you. Don't get my nephew into anything you can't get him out of.
[1159, 179, 39, 4, 1, 19, 38, 20, 4511, 105, 149, 1, 85, 38, 42, 36, 8]
I don't know if this would be a wise move...
[3, 19, 25, 48, 14, 94, 28, 5, 1788, 346]


In [13]:
maxlen=500
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_dev = sequence.pad_sequences(x_dev, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('x_dev shape:', x_dev.shape)

Pad sequences (samples x time)
('x_train shape:', (215353, 500))
('x_test shape:', (26920, 500))
('x_dev shape:', (26919, 500))


In [9]:
labels= train_y.values.reshape(-1,1)
labels_dev=dev_y.values.reshape(-1,1)
labels_test = test_y.values.reshape(-1,1)

In [ ]:
cnn_model = Sequential()
    
max_features=10000
embedding_dims=100

cnn_model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(10, 3, 
                     padding='valid', 
                     activation='relu',
                     strides=1,
                     input_shape=train.shape))

cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dropout(0.25))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(1))
          
cnn_model.add(Activation('sigmoid'))

cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
          
cnn_model.fit(x_train, labels,
          batch_size=32,
          epochs=4,
          validation_data=(x_dev, labels_dev))

Train on 215353 samples, validate on 26919 samples
Epoch 1/4
215353/215353 [==============================] - 297s 1ms/step - loss: 0.6107 - acc: 0.6949 - val_loss: 0.5954 - val_acc: 0.6995

In [14]:
score = cnn_model.evaluate(x_test, labels_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.58311944330252297)
('Test accuracy:', 0.70601783060921253)


In [12]:
cnn_model

In [ ]:
cnn_model = Sequential()
    
max_features=10000
embedding_dims=100

cnn_model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(10, 3, 
                     padding='valid', 
                     activation='relu',
                     strides=1,
                     input_shape=train.shape))

cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dropout(0.25))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(1))
          
cnn_model.add(Activation('softmax'))

cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
          
cnn_model.fit(x_train, labels,
          batch_size=32,
          epochs=4,
          validation_data=(x_dev, labels_dev))

Train on 215353 samples, validate on 26919 samples
Epoch 1/4
 29632/215353 [===>..........................] - ETA: 4:07 - loss: 4.8722 - acc: 0.6944 

In [20]:
score = cnn_model.evaluate(x_test, labels_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 4.8111418467574101)
('Test accuracy:', 0.69821693907875182)
